# 🧠 AI Agents Bootcamp - Day 3
## Interactive Interview Bot with LangGraph

**What you'll build:** A stateful, looping interview practice system that asks questions, analyzes your answers, gives feedback, and adapts.

**Key concepts:**
- LangGraph state machines
- Conditional routing (loops!)
- Typed state management
- Production patterns

**Time:** ~10 minutes setup, then practice as long as you want!

---

📚 **Full Series:** [Standout Systems on Substack](https://teodoracoach.substack.com/)  
💻 **GitHub:** [ai-agents-bootcamp](https://github.com/DoraSzasz/ai-agents-bootcamp)  
🎯 **Coaching:** [teodora.coach](https://teodora.coach)

## The Architecture

```
┌────────────────────────────────────────────────────────┐
│              INTERVIEW PRACTICE BOT                     │
├────────────────────────────────────────────────────────┤
│                                                        │
│  GENERATE ──▶ ASK ──▶ ANALYZE ──▶ FEEDBACK            │
│  QUESTIONS   QUESTION  ANSWER      │                   │
│                 ▲                   │                   │
│                 │                   ▼                   │
│                 └──── CONTINUE? ────┤                   │
│                       (loop)        │                   │
│                                     ▼                   │
│                                  WRAP UP ──▶ END       │
│                                                        │
└────────────────────────────────────────────────────────┘
```

**This loop is impossible in CrewAI.** LangGraph makes it elegant.

## Step 1: Install Dependencies

In [ ]:
# Install required packages
!pip install -q langgraph langchain-openai langchain-core

## Step 2: Set Up API Key

**Option A (Recommended):** Use Colab Secrets  
**Option B:** Paste directly below

In [ ]:
import os

# Try Colab secrets first
try:
    from google.colab import userdata
    os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
    print("✅ API key loaded from Colab Secrets")
except:
    # Fallback: paste your key here
    os.environ["OPENAI_API_KEY"] = "sk-your-key-here"  # ← Replace!
    print("⚠️  Using hardcoded key - add to Colab Secrets for security")

## Step 3: Import Libraries

In [ ]:
from typing import TypedDict, List, Literal, Annotated
from langgraph.graph import StateGraph, END
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
import operator
from datetime import datetime

print("✅ All libraries imported!")

## Step 4: Configure Your Interview

**🎯 CUSTOMIZE THIS for your actual interview!**

In [ ]:
# ═══════════════════════════════════════════════════════════
# 🎯 CUSTOMIZE THESE
# ═══════════════════════════════════════════════════════════

COMPANY = "Anthropic"              # Your target company
POSITION = "Senior ML Engineer"    # The role
DIFFICULTY = "medium"              # easy, medium, hard

print(f"🎯 Preparing for: {POSITION} at {COMPANY}")
print(f"📊 Difficulty: {DIFFICULTY}")

## Step 5: Define State (The Brain)

This is THE critical LangGraph concept. State is:
- **Typed** (catches bugs early)
- **Shared** (all nodes read/write)
- **Persistent** (can save/resume)

In [ ]:
class InterviewState(TypedDict):
    """Everything the bot knows and remembers."""
    
    # Configuration
    company: str
    position: str
    difficulty: str
    
    # Questions
    questions: List[str]
    current_index: int
    
    # Conversation (Annotated = auto-append)
    exchanges: Annotated[List[dict], operator.add]
    
    # Performance
    scores: List[int]
    weak_areas: List[str]
    
    # Flow control
    user_wants_continue: bool
    session_complete: bool

print("✅ State schema defined")

## Step 6: Build the Nodes

Each node is a function: `state in → updated state out`

In [ ]:
# Initialize LLM
llm = ChatOpenAI(model="gpt-4o", temperature=0.7)

# ═══════════════════════════════════════════════════════════
# NODE 1: GENERATE QUESTIONS
# ═══════════════════════════════════════════════════════════
def generate_questions(state: InterviewState) -> dict:
    """Generate tailored interview questions."""
    
    print("\n🎯 Generating questions...")
    
    response = llm.invoke([
        SystemMessage(content=f"""You are a senior interviewer at {state['company']}.
        Generate exactly 5 interview questions for a {state['position']} role.
        Difficulty: {state['difficulty']}
        
        Mix: 2 technical, 2 behavioral, 1 scenario.
        Return ONLY numbered questions 1-5."""),
        HumanMessage(content="Generate questions now.")
    ])
    
    # Parse questions
    lines = response.content.strip().split('\n')
    questions = [l.strip() for l in lines if l.strip() and l[0].isdigit()]
    
    print(f"\n✅ Generated {len(questions)} questions\n")
    for q in questions:
        print(f"   {q[:65]}..." if len(q) > 65 else f"   {q}")
    
    return {
        "questions": questions[:5],
        "current_index": 0,
        "exchanges": [],
        "scores": [],
        "weak_areas": []
    }


# ═══════════════════════════════════════════════════════════
# NODE 2: ASK QUESTION
# ═══════════════════════════════════════════════════════════
def ask_question(state: InterviewState) -> dict:
    """Present question and collect answer."""
    
    idx = state["current_index"]
    question = state["questions"][idx]
    
    print("\n" + "=" * 55)
    print(f"📝 QUESTION {idx + 1} of {len(state['questions'])}")
    print("=" * 55)
    print(f"\n🎤 {question}\n")
    
    # Get answer
    user_answer = input("Your answer: ")
    
    exchange = {
        "question_num": idx + 1,
        "question": question,
        "answer": user_answer
    }
    
    return {"exchanges": [exchange]}


# ═══════════════════════════════════════════════════════════
# NODE 3: ANALYZE ANSWER
# ═══════════════════════════════════════════════════════════
def analyze_answer(state: InterviewState) -> dict:
    """Score and analyze the answer."""
    
    last = state["exchanges"][-1]
    print("\n⏳ Analyzing...")
    
    response = llm.invoke([
        SystemMessage(content="""Analyze this interview answer.
        
        Format EXACTLY:
        SCORE: [1-10]
        STRENGTHS: [what was good]
        IMPROVEMENTS: [what to improve]
        PRO TIP: [one tip]
        WEAK_AREA: [skill to work on, or "none" if score >= 7]"""),
        HumanMessage(content=f"Q: {last['question']}\nA: {last['answer']}")
    ])
    
    # Parse score
    content = response.content
    score = 5
    weak_area = None
    
    for line in content.split('\n'):
        if line.startswith('SCORE:'):
            try:
                score = int(line.split(':')[1].strip().split('/')[0])
            except: pass
        elif line.startswith('WEAK_AREA:'):
            area = line.split(':')[1].strip()
            if area.lower() != 'none':
                weak_area = area
    
    state["exchanges"][-1]["score"] = score
    state["exchanges"][-1]["feedback"] = content
    
    result = {"scores": state.get("scores", []) + [score]}
    if weak_area:
        result["weak_areas"] = state.get("weak_areas", []) + [weak_area]
    
    return result


# ═══════════════════════════════════════════════════════════
# NODE 4: GIVE FEEDBACK
# ═══════════════════════════════════════════════════════════
def give_feedback(state: InterviewState) -> dict:
    """Show feedback and ask to continue."""
    
    last = state["exchanges"][-1]
    score = last.get("score", 5)
    feedback = last.get("feedback", "")
    
    print("\n" + "=" * 55)
    print("📊 FEEDBACK")
    print("=" * 55)
    
    bar = "█" * score + "░" * (10 - score)
    emoji = "🌟" if score >= 8 else "✅" if score >= 6 else "🔧"
    print(f"\n{emoji} Score: [{bar}] {score}/10\n")
    print(feedback)
    
    new_index = state["current_index"] + 1
    remaining = len(state["questions"]) - new_index
    
    if remaining > 0:
        cont = input(f"\n➡️  Continue? ({remaining} left) (yes/no): ").lower()
        wants_continue = cont in ['yes', 'y', '']
    else:
        print("\n📝 All questions done!")
        wants_continue = False
    
    return {
        "current_index": new_index,
        "user_wants_continue": wants_continue
    }


# ═══════════════════════════════════════════════════════════
# NODE 5: WRAP UP
# ═══════════════════════════════════════════════════════════
def wrap_up(state: InterviewState) -> dict:
    """Final summary and advice."""
    
    print("\n" + "=" * 55)
    print("🎓 SESSION COMPLETE")
    print("=" * 55)
    
    scores = state.get("scores", [])
    if scores:
        avg = sum(scores) / len(scores)
        print(f"\n📊 Results:")
        print(f"   • Questions: {len(scores)}")
        print(f"   • Average: {avg:.1f}/10")
        print(f"   • Best: {max(scores)}/10")
    
    if state.get("weak_areas"):
        print(f"\n🎯 Practice these:")
        for area in set(state["weak_areas"]):
            print(f"   • {area}")
    
    # Final advice
    response = llm.invoke([
        SystemMessage(content="""Give 2 specific tips based on this session.
        Keep under 75 words. Be encouraging."""),
        HumanMessage(content=f"Scores: {scores}, Weak areas: {state.get('weak_areas', [])}")
    ])
    
    print(f"\n💡 Coach's Notes:\n{response.content}")
    
    return {"session_complete": True}


print("✅ All 5 nodes defined!")

## Step 7: The Router (Decision Logic)

This is what makes LangGraph special: **conditional edges**

In [ ]:
def should_continue(state: InterviewState) -> Literal["ask_question", "wrap_up"]:
    """Decide: loop back or exit?"""
    
    # User said stop
    if not state.get("user_wants_continue", True):
        return "wrap_up"
    
    # All questions done
    if state["current_index"] >= len(state["questions"]):
        return "wrap_up"
    
    # Continue the loop!
    return "ask_question"

print("✅ Router defined")

## Step 8: Assemble the Graph

In [ ]:
# Create the graph
workflow = StateGraph(InterviewState)

# Add nodes
workflow.add_node("generate_questions", generate_questions)
workflow.add_node("ask_question", ask_question)
workflow.add_node("analyze_answer", analyze_answer)
workflow.add_node("give_feedback", give_feedback)
workflow.add_node("wrap_up", wrap_up)

# Set entry
workflow.set_entry_point("generate_questions")

# Linear edges
workflow.add_edge("generate_questions", "ask_question")
workflow.add_edge("ask_question", "analyze_answer")
workflow.add_edge("analyze_answer", "give_feedback")

# THE LOOP - conditional edge
workflow.add_conditional_edges(
    "give_feedback",
    should_continue,
    {
        "ask_question": "ask_question",  # Loop!
        "wrap_up": "wrap_up"
    }
)

workflow.add_edge("wrap_up", END)

# Compile
app = workflow.compile()

print("✅ Graph compiled and ready!")

## Step 9: Visualize the Graph (Optional)

In [ ]:
# Visualize the flow (requires graphviz)
try:
    from IPython.display import Image, display
    display(Image(app.get_graph().draw_mermaid_png()))
except:
    print("Graph visualization requires: pip install grandalf")
    print("\nFlow: generate → ask → analyze → feedback → [loop or wrap_up] → END")

## Step 10: Run Your Interview Bot! 🚀

In [ ]:
# Initialize state
initial_state = {
    "company": COMPANY,
    "position": POSITION,
    "difficulty": DIFFICULTY,
    "questions": [],
    "current_index": 0,
    "exchanges": [],
    "scores": [],
    "weak_areas": [],
    "user_wants_continue": True,
    "session_complete": False
}

print("=" * 55)
print("🤖 INTERVIEW PRACTICE BOT")
print("=" * 55)
print(f"📍 Company: {COMPANY}")
print(f"💼 Position: {POSITION}")
print(f"📊 Difficulty: {DIFFICULTY}")
print("=" * 55)

# Run it!
result = app.invoke(initial_state)

print("\n✅ Session complete! Good luck! 🍀")

## Step 11: Review Your Session

In [ ]:
from IPython.display import Markdown, display

# Build summary
summary = f"""## 📋 Session Summary

**Company:** {result['company']}  
**Position:** {result['position']}  
**Questions Completed:** {len(result['scores'])}

### Scores
"""

for i, (ex, score) in enumerate(zip(result['exchanges'], result['scores']), 1):
    bar = "█" * score + "░" * (10 - score)
    summary += f"- Q{i}: [{bar}] {score}/10\n"

if result['weak_areas']:
    summary += "\n### Areas to Practice\n"
    for area in set(result['weak_areas']):
        summary += f"- {area}\n"

display(Markdown(summary))

---

## 🎯 Experiments to Try

**Easy:**
- Change company/position to your actual interview
- Try different difficulty levels

**Medium:**
- Add a "hint" option before answering
- Track time spent per question
- Save results to a file

**Advanced:**
- Add voice input (speech-to-text)
- Make questions adaptive (harder if doing well)
- Add a "mock interviewer" that asks follow-ups

---

## 🏆 The Complete Journey

| Day | What You Learned | Pattern |
|-----|-----------------|--------|
| 1 | Agent fundamentals | Single agent, basic loop |
| 2 | Multi-agent systems | CrewAI, task chaining |
| **3** | **Production patterns** | **LangGraph, state, loops** |

**You now have the complete toolkit for building real AI systems.**

---

## 📚 Keep Learning

**Subscribe:** [Standout Systems](https://teodoracoach.substack.com/)  
**GitHub:** [ai-agents-bootcamp](https://github.com/DoraSzasz/ai-agents-bootcamp)  
**Coaching:** [teodora.coach](https://teodora.coach)